Import mmsdk package:

In [7]:
from constants import SDK_PATH, DATA_PATH, WORD_EMB_PATH, CACHE_PATH
import sys

if SDK_PATH is None:
    print("SDK path is not specified! Please specify first in constants/paths.py")
    exit(0)
else:
    sys.path.append(SDK_PATH)

import mmsdk
import os
import re
import numpy as np
from mmsdk import mmdatasdk as md
from subprocess import check_call, CalledProcessError

# create folders for storing the data
if not os.path.exists(DATA_PATH):
    check_call(' '.join(['mkdir', '-p', DATA_PATH]), shell=True)

In [8]:
DATASET = md.cmu_mosei
all_num = DATASET.standard_folds.standard_train_fold + \
     DATASET.standard_folds.standard_test_fold + \
     DATASET.standard_folds.standard_valid_fold
print('all', len(all_num))
print('train', len(DATASET.standard_folds.standard_train_fold))
print('valid', len(DATASET.standard_folds.standard_valid_fold))
print('test', len(DATASET.standard_folds.standard_test_fold))


all 3227
train 2249
valid 300
test 678


Preparation import to modify mmsdk dataset alignment method

In [9]:
from mmsdk.mmdatasdk import log, computational_sequence
import numpy
import time
import struct
from tqdm import tqdm_notebook as tqdm

Inheritate from mmsdk with new methods: 

In [10]:

def save_htk_format(features, feature_type, folder, video_number):
    '''
    This function works for function align_upsampling_and_save. It save feature vectors of one video id into one 
    htk format file. 
    feature: feature of one video
    All files are of USER type.
    number of samples being consistent with number of samples in csd file
    sample bytes for COVAREP: 4 * 74 = 296
    sample period: 10 000.0 us (100000)
    paramkind: USER (9)
    '''
    #set default extension
    ext = '.txt'
    file_name = os.path.join(folder, feature_type, video_number)

    num_sample = len(features)
    byte_n_sample = num_sample.to_bytes(4, byteorder='big')

    period = 100000
    byte_period = period.to_bytes(4, byteorder = 'big')

    if feature_type == 'COVAREP':
        sample_b = 296
        ext = '.cov'
    elif feature_type == 'WordVec':
        sample_b = 1200
        ext = '.wvec'

    byte_sample_b = sample_b.to_bytes(2, byteorder = 'big')

    sample_type = 9
    byte_sample_type = sample_type.to_bytes(2, byteorder = 'big')
    header = byte_n_sample + byte_period + byte_sample_b + byte_sample_type

    output_byte = b''
    '''
    try:
        print(type(features))
        print(type(features[0]))
    except:
        print('features cannot be indexed')
        
    try:
        print(features[0].shape)
    except:
        print('features[0] is not np array')
    '''
    for datapoint in features:
        y = list(map(lambda x: struct.pack('>f', x), datapoint))
        byte_datapoint = b''.join(y)

        output_byte += byte_datapoint

    with open(file_name + ext, 'wb') as file:
        file.write(header + output_byte)



In [11]:

import pickle, os, json, codecs


def save_intervals(intervals, feature_type, folder, video_number):
    #set default extension
    ext = '.json'
    video_number = 'intervals_' + video_number
    file_name = os.path.join(folder, feature_type, video_number)
    
    if type(intervals) != list:
        intervals = intervals.tolist()
    json.dump(intervals, codecs.open(file_name + ext, 'w', encoding='utf-8'), indent=4)
    

In [12]:
class new_mmdataset(md.mmdataset):
    #TODO: Need tqdm bar for this as well
    def get_relevant_entries(self,reference):
        relevant_entries={}
        relevant_entries_np={}

        #pbar = tqdm(total=count,unit=" Computational Sequence Entries",leave=False)

        #otherseq_key: OpenFace, wordvec, etc
        for otherseq_key in set(list(self.computational_sequences.keys()))-set([reference]):
            relevant_entries[otherseq_key]={}
            relevant_entries_np[otherseq_key]={}
            sub_compseq=self.computational_sequences[otherseq_key]
            # for some_id in all video ids
            for key in list(sub_compseq.data.keys()):
                keystripped=key.split('[')[0]
                if keystripped not in relevant_entries[otherseq_key]:                           
                    relevant_entries[otherseq_key][keystripped]={}
                    relevant_entries[otherseq_key][keystripped]["intervals"]=[]                     
                    relevant_entries[otherseq_key][keystripped]["features"]=[]                                                            
                    
                relev_intervals=self.computational_sequences[otherseq_key].data[key]["intervals"]                                             
                relev_features=self.computational_sequences[otherseq_key].data[key]["features"]         
                if len(relev_intervals.shape)<2:
                    relev_intervals=relev_intervals[None,:]
                    relev_features=relev_features[None,:]

                relevant_entries[otherseq_key][keystripped]["intervals"].append(relev_intervals)
                relevant_entries[otherseq_key][keystripped]["features"].append(relev_features)
                
            for key in list(relevant_entries[otherseq_key].keys()):
                relev_intervals_np=numpy.concatenate(relevant_entries[otherseq_key][key]["intervals"],axis=0)                                 
                relev_features_np=numpy.concatenate(relevant_entries[otherseq_key][key]["features"],axis=0)
                sorted_indices=sorted(range(relev_intervals_np.shape[0]),key=lambda x: relev_intervals_np[x,0])                               
                relev_intervals_np=relev_intervals_np[sorted_indices,:]                         
                relev_features_np=relev_features_np[sorted_indices,:]

                relevant_entries_np[otherseq_key][key]={}
                relevant_entries_np[otherseq_key][key]["intervals"]=relev_intervals_np
                relevant_entries_np[otherseq_key][key]["features"]=relev_features_np
            log.status("Pre-alignment done for <%s> ..."%otherseq_key)
        return relevant_entries_np
    
    def intersect_and_copy_upsampling(self, ref_all, relevant_entry, not_enough_label, epsilon, log_file, feature_info):
        #ref_all: reference_entries[other_key][entry_key]['intervals'] e.g. [COVAREP][some video id]['intervals']
        #relevant_entry: relevant_entries[other_key][entry_key] e.g. [COVAREP][some video id]
        #epsilon: error allowed in alignment
        #ref_time < one interval in relevant_entry
        
        pbar_small=log.progress_bar(total=ref_all.shape[0],unit=" Segments",leave=False)
        pbar_small.set_description("Aligning: " + feature_info)
        
        sub=relevant_entry["intervals"]
        features=relevant_entry["features"]
        
        #finding where intersect happens
        pointer_b = 0 # for relevant_entry
        aligned_sub = []
        aligned_feature = []
        
        for i, inter in enumerate(ref_all):
            #print(pointer_b)
            if (abs(inter[0]-inter[1])<epsilon):
                pbar_small.update(1)
                continue
            pointer_c = pointer_b
            while(pointer_c < sub.shape[0]):
                if (inter[0] - sub[pointer_c][0]) > (-epsilon) and (sub[pointer_c][1] - inter[0]) > (-epsilon):
                    aligned_sub.append(sub[pointer_c])
                    aligned_feature.append(features[pointer_c])
                    break
                else:
                    pointer_c += 1
                
            if pointer_c == sub.shape[0]:
                diff = list(map(lambda x: abs(inter[0] - x), sub[:, 0]))
                min_diff = min(diff)
                pointer_c = diff.index(min_diff)
                with open(log_file, 'w+') as fi:
                    fi.write('no corresponding frame, find the closest one, {}, difference: {}\n'.format(feature_info, min_diff))
                aligned_sub.append(sub[pointer_c])
                aligned_feature.append(features[pointer_c])
            else:
                pointer_b = pointer_c

            pbar_small.update(1)
        
        aligned_sub = np.array(aligned_sub)
        aligned_feature = np.array(aligned_feature)
        zero_idx = np.where(np.isinf(aligned_feature))
        aligned_feature[zero_idx] = 0

        pbar_small.close()
        return aligned_sub,aligned_feature
    
    def align_upsampling_and_save(self, reference, id_idx, label_dataset, collapse_function=None, epsilon = 10e-6):
        folder = '/data/mifs_scratch/yw454/cmumosei_aligned'
        log_file = './mosei_alignment_log.txt'
        not_enough_label_file = './mosei_notenough_lable_videos.txt'
        no_valid_label_file = './mosei_no_lable_videos.txt'
        
        #aligned_output = {}
        count = 0
        
        ##self.computational_sequences.keys are COVERAP, OpenFace, WordVec, etc
        #for sequence_name in self.computational_sequences.keys():
        #    #init a dictionary to store different featues seperately
        #    aligned_output[sequence_name]={}
        
        if reference not in self.computational_sequences.keys():
            log.error("Computational sequence <%s> does not exist in dataset"%reference,error=True)
        
        #get data of reference feature
        refseq=self.computational_sequences[reference].data
        #unifying the dataset, removing any entries that are not in the reference computational sequence
        self.unify()
        
        #building the relevant entries to the reference - what we do in this section is simply removing all the [] from the entry ids and populating them into a new dictionary
        log.status("Pre-alignment based on <%s> computational sequence started ..."%reference)
        
        relevant_entries=self.get_relevant_entries(reference)
        log.status("Alignment starting ...")
        
        
        pbar = log.progress_bar(total=len(refseq.keys()),unit=" Computational Sequence Entries",leave=False)
        pbar.set_description("Overall Progress")
        # for some_id in all video ids
        for entry_key in list(refseq.keys()):
            not_enough_label = False
            if entry_key in id_idx:
                stored_idx = id_idx.index(entry_key)
                if stored_idx < 104 or (stored_idx > 104 and stored_idx < 1781):
                #if stored_idx != 1781:
                    continue
            
            label_len = label_dataset[entry_key]['intervals'][-1][1] - label_dataset[entry_key]['intervals'][0][0]
            reference_len = refseq[entry_key]['intervals'][-1][1] - refseq[entry_key]['intervals'][0][0]
            div = reference_len / label_len
            if div > 2:
                not_enough_label = True
                with open(not_enough_label_file, 'w+') as fw:
                    fw.write(entry_key + '\n')
            if label_len < 0:
                with open(no_valid_label_file, 'w+') as fw:
                    fw.write(entry_key + '\n')
                
            all_intersects = {}
            all_intersect_features = {}
            
            #for sequence_name in self.computational_sequences.keys():
            #    all_intersects[sequence_name] = []
            #    all_intersect_features[sequence_name] = []
            
            ref_all=refseq[entry_key]['intervals']
                
            #aligning all sequences to ref sequence (previous: align refer to refer as well, now: not include refer)
            #otherseq_key: other features; entry_key: some video id
                
            for otherseq_key in list(self.computational_sequences.keys()):
                if otherseq_key != reference:
                    feature_info = 'reference: {}, other feature {}, video id: {}'.format(reference, otherseq_key, entry_key)
                    intersects,intersects_features=self.intersect_and_copy_upsampling(ref_all,relevant_entries[otherseq_key][entry_key], not_enough_label, epsilon, log_file, feature_info)
                else:
                    intersects,intersects_features=refseq[entry_key]['intervals'][:,:],refseq[entry_key]['features'][:,:]
                    
                #print(type(intersects[0]))
                #print(type(intersects_features[0]))
                #print(len(intersects[0]))
                #print(len(intersects_features[0]))
                all_intersects[otherseq_key] = intersects
                all_intersect_features[otherseq_key] = intersects_features
                    

            #save features per video
            for sequence_name in self.computational_sequences.keys():
                video_code = id_idx.index(entry_key)
                video_code = str(video_code).zfill(6)
                
                save_htk_format(all_intersect_features[sequence_name], sequence_name, folder, video_code)
                save_intervals(all_intersects[sequence_name], sequence_name, folder, video_code)
                print('alignment saved for video {} feature {}.'.format(video_code, sequence_name))
            
            pbar.update(1)
        pbar.close()




In [13]:
# define your different modalities - refer to the filenames of the CSD files
basic_dict={'COVAREP': DATA_PATH + 'CMU_MOSEI_COVAREP.csd', 
            'WordVec': DATA_PATH + 'CMU_MOSEI_TimestampedWordVectors.csd'}
second_dict = {'Facet': DATA_PATH + 'CMU_MOSEI_VisualFacet42.csd',
            'OpenFace': DATA_PATH + 'CMU_MOSEI_VisualOpenFace2.csd'}
other_dict = {'Word': DATA_PATH + 'CMU_MOSEI_TimestampedWords.csd',
             'Phone': DATA_PATH + 'CMU_MOSEI_TimestampedPhones.csd'}
label_dict = {'mylabels':DATA_PATH + 'CMU_MOSEI_Labels.csd'}

In [14]:
# list the directory contents... let's see what features there are
data_files = os.listdir(DATA_PATH)
print('\n'.join(data_files))

CMU_MOSEI_VisualOpenFace2.csd
CMU_MOSEI_TimestampedWords.csd
CMU_MOSEI_COVAREP.csd
CMU_MOSEI_VisualFacet42.csd
CMU_MOSEI_TimestampedWordVectors.csd
CMU_MOSEI_TimestampedPhones.csd
CMU_MOSEI_Labels.csd
COVAREP.zip


In [15]:
basic_dataset = new_mmdataset(basic_dict)
second_dataset = new_mmdataset(second_dict)
label_dataset = new_mmdataset(label_dict)

validators eliminated
[2021-01-29 16:06:19.612] | Success | Computational sequence read from file /data/mifs_scratch/yw454/cmu_mosei_latest/CMU_MOSEI_COVAREP.csd ...
[2021-01-29 16:06:20.447] | Warning | Validation of the computational sequence skipped by user request.
validators eliminated
[2021-01-29 16:06:20.449] | Success | Computational sequence read from file /data/mifs_scratch/yw454/cmu_mosei_latest/CMU_MOSEI_TimestampedWordVectors.csd ...
[2021-01-29 16:06:20.689] | Warning | Validation of the computational sequence skipped by user request.
[2021-01-29 16:06:20.690] | Success | Dataset initialized successfully ... 
validators eliminated
[2021-01-29 16:06:20.692] | Success | Computational sequence read from file /data/mifs_scratch/yw454/cmu_mosei_latest/CMU_MOSEI_VisualFacet42.csd ...
[2021-01-29 16:06:20.920] | Warning | Validation of the computational sequence skipped by user request.
validators eliminated
[2021-01-29 16:06:20.922] | Success | Computational sequence read from 

In [16]:
[COVAREP, WordVec] = [basic_dataset.computational_sequences['COVAREP'],
                        basic_dataset.computational_sequences['WordVec']]
label = label_dataset.computational_sequences['mylabels']
OpenFace = second_dataset.computational_sequences['OpenFace']

In [23]:
some_id = list(label.keys())[1]
if some_id in all_num:
    print(label[some_id]['intervals'][:, :])

[[ 82.753 100.555]
 [119.919 125.299]
 [  4.84   14.052]
 [ 13.211  27.521]
 [ 26.541  41.3  ]
 [ 74.083  82.776]]


In [13]:
utterance = 0
yesterday = 0
for i, video in enumerate(list(WordVec.keys())):
    utterance += WordVec[video]['intervals'].shape[0]
    if i > 2132 and i <= 2270: # and i >= 780: 
        yesterday += WordVec[video]['intervals'].shape[0]
            
print(yesterday / utterance)

0.03206996277692446


In [12]:
'''
weird_acoustic = []
norm_acoustic = []
for i, k in enumerate(list(COVAREP.keys())):
    interval = COVAREP[k]['intervals']
    end_frame = int((interval[-1][0] - interval[0][0]) * 100)
    if abs(end_frame - (len(interval) - 1)) > 3:
        weird_acoustic.append(k)
        #print(end_frame, (len(interval) - 1))
    else:
        norm_acoustic.append(k)
    if i == 0:
        print(interval[:, :])
        
      
print(len(weird_acoustic))
#print(len(norm_acoustic))
if len(weird_acoustic) > 2:
    for i in range(2):
        interval = COVAREP[weird_acoustic[i]]['intervals'][:, :]

        for j in range(len(interval) - 1):

            if abs(interval[j][1] - interval[j+1][0]) > 1e-6:
                print(j, interval[j][1], interval[j+1][0])
            

            if abs(interval[j][1] - interval[j][0] - 0.01) >  1e-6:
                print(j, interval[j][1], interval[j][0])
            

        print(COVAREP[weird_acoustic[i]]['intervals'][-1])
        print(COVAREP[weird_acoustic[i]]['intervals'][0])

        print(int((interval[-1][0] - interval[0][0]) * 100))
        print(len(COVAREP[weird_acoustic[i]]['intervals']) - 1)

'''    
    

"\nweird_acoustic = []\nnorm_acoustic = []\nfor i, k in enumerate(list(COVAREP.keys())):\n    interval = COVAREP[k]['intervals']\n    end_frame = int((interval[-1][0] - interval[0][0]) * 100)\n    if abs(end_frame - (len(interval) - 1)) > 3:\n        weird_acoustic.append(k)\n        #print(end_frame, (len(interval) - 1))\n    else:\n        norm_acoustic.append(k)\n    if i == 0:\n        print(interval[:, :])\n        \n      \nprint(len(weird_acoustic))\n#print(len(norm_acoustic))\nif len(weird_acoustic) > 2:\n    for i in range(2):\n        interval = COVAREP[weird_acoustic[i]]['intervals'][:, :]\n\n        for j in range(len(interval) - 1):\n\n            if abs(interval[j][1] - interval[j+1][0]) > 1e-6:\n                print(j, interval[j][1], interval[j+1][0])\n            \n\n            if abs(interval[j][1] - interval[j][0] - 0.01) >  1e-6:\n                print(j, interval[j][1], interval[j][0])\n            \n\n        print(COVAREP[weird_acoustic[i]]['intervals'][-1])\n 

In [16]:
wordvec_with_gap = {}
not_zeros_start = set()
#video = list(WordVec.keys())[0]
for k in list(WordVec.keys()):
    interval = WordVec[k]['intervals']
    #if k == video:
    #    print(interval[:, :])
    if int(np.floor(interval[0][0])) != 0:
        not_zeros_start.add(k)
        print(interval[0][0])
    for i in range(interval.shape[0] - 1):
        #if k == video:
        #    print(interval[i+1][0], interval[i][1])
        if abs(interval[i + 1][0] - interval[i][1]) > 1e-3:
            if k in wordvec_with_gap:
                wordvec_with_gap[k] += 1
            else:
                wordvec_with_gap[k] = 1

print(wordvec_with_gap)
print(len(not_zeros_start))

{}
0


In [14]:
not_enough_word = set()
not_enough_label = set()
error_video = set()
no_labels = set()
label_keys = set(label.keys())
long_video_file = './video_short_label.txt'

for k in list(COVAREP.keys()):
    acoustic = COVAREP[k]['intervals'][-1][1]
    language = WordVec[k]['intervals'][-1][1]
    
    if language / acoustic > 2 or acoustic / language > 2 or language / acoustic < 0:
        not_enough_word.add(k)
    
    if k in label_keys:
        annotations = label[k]['intervals'][-1][1] - label[k]['intervals'][0][0]
        
        if annotations < 0: # annotations / acoustic > 2 or acoustic / annotations > 2 or 
        #if annotations - language > 1:
            if k in all_num:
                not_enough_label.add(k)
                #print(COVAREP[k]['intervals'][0][0], label[k]['intervals'][0][0])
                #print(acoustic, annotations)
                #print(k)
                #print('acoustic:', COVAREP[k]['intervals'][-1][-1], 'label', label[k]['intervals'][-1][-1])
            else:
                error_video.add(k)
                
                #with open(long_video_file, 'w+') as f_w:
                #    f_w.write(k + '\n')
            
    else:
        no_labels.add(k)
'''
print('========================================') 
print(len(not_enough_label))
print(len(not_enough_word))
print(len(error_video))

print(len((not_enough_label & not_enough_word)))

print(len(no_labels))  
print(len(no_labels & set(all_num)))
print(len((label_keys - not_enough_label) & not_enough_word))
'''   

for i, video in enumerate(not_enough_label):
    if i < 2:
        print(video)
        print('COVAREP last interval', COVAREP[video]['intervals'][-1][1])
        print('WordVec last interval', WordVec[video]['intervals'][-1][1])
        print(label[video]['intervals'][-1][1])
        print(label[video]['intervals'][:, :])
        #annotations = label[video]['intervals'][-1][1] - label[k]['intervals'][0][0]
        #acoustic = COVAREP[video]['intervals'][-1][1]
        #print(acoustic / annotations)

        
     


ZDz8Qr-sJ3E
COVAREP last interval 119.35000000000001
WordVec last interval 119.321315193
73.217
[[80.728 90.987]
 [14.159 23.041]
 [22.041 26.483]
 [67.638 73.217]]
trQLgl6ncmk
COVAREP last interval 98.17
WordVec last interval 98.1394558821
59.818
[[74.881 80.66 ]
 [79.66  96.564]
 [13.98  20.128]
 [31.25  44.912]
 [53.709 59.818]]


In [21]:
video = 'c7xUcM68IFE'
print(video in list(COVAREP.keys()))
print(video in label_keys)
annotations = label[k]['intervals'][-1][1] - label[k]['intervals'][0][0]
acoustic = COVAREP[k]['intervals'][-1][1]
print(annotations / acoustic > 2)
print(acoustic / annotations > 2)
print(annotations / acoustic > 2 or acoustic / annotations > 2)

True
True
False
True
True


In [23]:
video = '-NFrJFQijFE'
print(label[video]['intervals'][:10])
print(COVAREP[video]['intervals'][-1][1], COVAREP[video]['intervals'][0][0])
print(OpenFace[video]['intervals'][-1][1], OpenFace[video]['intervals'][0][0])

[[-0.47755102  4.10430839]
 [ 3.37369615  9.02312925]
 [ 8.02312925 14.72018141]
 [44.11111111 54.769161  ]]
570.0 0.0
569.92 0.0


In [44]:
video_id_record = os.path.join('/data/mifs_scratch/yw454/cmumosei_aligned', 'video_id.json')

with open(video_id_record, 'r') as json_file:
    video_ids = json.load(json_file)

basic_dataset.align_upsampling_and_save('COVAREP', video_ids)



[2021-01-24 02:53:58.267] | Status  | Unify was called ...
[2021-01-24 02:53:58.277] | Warning | PEBwwe0PLZ8 entry is not shared among all sequences, removing it ...
[2021-01-24 02:53:58.280] | Success | Unify completed ...
[2021-01-24 02:53:58.280] | Status  | Pre-alignment based on <COVAREP> computational sequence started ...










  0%|          | 0/3836 [00:00<?, ? Computational Sequence Entries/s]







Overall Progress:   0%|          | 0/3836 [00:00<?, ? Computational Sequence Entries/s]








  0%|          | 0/5459 [00:00<?, ? Segments/s]








Aligning: reference: COVAREP, other feature WordVec, video id: HUOMbK1x7MI:   0%|          | 0/5459 [00:00<?, ? Segments/s]

[2021-01-24 02:54:19.297] | Status  | Pre-alignment done for <WordVec> ...
[2021-01-24 02:54:20.331] | Status  | Alignment starting ...
(5459, 2)
(5459, 74)











Aligning: reference: COVAREP, other feature WordVec, video id: HUOMbK1x7MI:   9%|▉         | 513/5459 [00:00<00:00, 5128.32 Segments/s]








Aligning: reference: COVAREP, other feature WordVec, video id: HUOMbK1x7MI:  22%|██▏       | 1212/5459 [00:00<00:00, 5572.12 Segments/s]








Aligning: reference: COVAREP, other feature WordVec, video id: HUOMbK1x7MI:  35%|███▌      | 1926/5459 [00:00<00:00, 5965.00 Segments/s]








Aligning: reference: COVAREP, other feature WordVec, video id: HUOMbK1x7MI:  48%|████▊     | 2647/5459 [00:00<00:00, 6289.07 Segments/s]








Aligning: reference: COVAREP, other feature WordVec, video id: HUOMbK1x7MI:  62%|██████▏   | 3361/5459 [00:00<00:00, 6520.21 Segments/s]








Aligning: reference: COVAREP, other feature WordVec, video id: HUOMbK1x7MI:  75%|███████▍  | 4084/5459 [00:00<00:00, 6717.07 Segments/s]








Aligning: reference: COVAREP, other feature WordVec, video id: HUOMbK1x7MI:  88%|████████▊ | 4807/5459 [00:00<00:00, 6861.

COVAREP
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(74,)
alignment saved for video 001815.
WordVec
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(300,)










Overall Progress:   0%|          | 1/3836 [00:06<6:51:32,  6.44s/ Computational Sequence Entries]







                                                                                                 

alignment saved for video 001815.


In [47]:
print(COVAREP['HUOMbK1x7MI']['features'].shape)

(5459, 74)
